In [1]:
import pandas as pd

In [4]:
questions_df = pd.read_parquet("../data/intermediate/cleaned_arena_data/questions.parquet")
threads_df = pd.read_parquet("../data/intermediate/cleaned_arena_data/threads.parquet")

In [3]:
questions_df

,question_id,thread_id,turn_number,user_query,question_role
0,q_00000000,thread_00000000,1,what models are you?,user
1,q_00000001,thread_00000001,1,تجم تفسرلي علاش فما صبع زايد في مرض blackfond ...,user
2,q_00000002,thread_00000002,1,Ellen White über die Flache Erde,user
3,q_00000003,thread_00000003,1,Test,user
4,q_00000004,thread_00000004,1,"Site: ""m.egwwritings.org/en"" über die flache Erde",user
...,...,...,...,...,...
32879,q_00032879,thread_00024066,6,EPDM기반의 식을 알려줘,user
32880,q_00032880,thread_00024067,1,Какой маленький бизнес открыть в новосибирске?,user
32881,q_00032881,thread_00024067,2,А какой можно начать дома с минимальными затра...,user
32882,q_00032882,thread_00024067,3,Сделай анализ по авито,user


In [29]:
questions_merged_df = threads_df.merge(questions_df[['thread_id', 'question_id', 'turn_number', 'user_query']], on='thread_id')

In [30]:
questions_merged_df.primary_intent.value_counts()

primary_intent
Factual Lookup         6015
Info Synthesis         5763
Analysis               3568
Explanation            3415
Recommendation         3357
Guidance               3357
Creative Generation    3264
Other                  3044
Text Processing        1100
Name: count, dtype: int64

In [31]:
questions_merged_df.client_country.value_counts()

client_country
US    5134
RU    3197
DE    1862
CN    1401
VN    1141
      ... 
MZ       1
MT       1
MG       1
TG       1
MQ       1
Name: count, Length: 139, dtype: int64

In [44]:
filtered_questions_df = questions_merged_df[questions_merged_df.languages.apply(lambda x: len(x) == 1 and 'English' in x)].query("client_country == 'US'")

In [46]:
filtered_questions_df.user_query.value_counts()

user_query
Who are you? Respond with only your name and who trained you.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [16]:
questions_merged_df.query("client_country == 'US'")

,thread_id,original_row_id,timestamp,total_turns,winner,judge,primary_intent,secondary_intent,languages,client_country,turn_number,user_query
4472,thread_00003377,3377,2025-03-25 19:55:49.639,1,tie,1bd116a77d2ab4795609e89cd76609ff317efe1e28fe20...,Recommendation,Unassigned,[English],US,1,what's the best boba tea at Berkeley? Answer i...
4473,thread_00003378,3378,2025-03-25 19:57:01.891,2,tie,1bd116a77d2ab4795609e89cd76609ff317efe1e28fe20...,Recommendation,Unassigned,[English],US,1,what's the best boba tea at Berkeley? Answer i...
4474,thread_00003379,3379,2025-03-25 19:57:38.867,1,None,a686fdd7534caae21ec28eb3b2fc0fa16348707823d660...,Other,Unassigned,[English],US,1,where am i
4475,thread_00003380,3380,2025-03-25 19:57:48.312,1,tie,1bd116a77d2ab4795609e89cd76609ff317efe1e28fe20...,Factual Lookup,Unassigned,[English],US,1,where am I right now in a country level
4476,thread_00003381,3381,2025-03-25 19:57:59.353,1,None,a686fdd7534caae21ec28eb3b2fc0fa16348707823d660...,Factual Lookup,Unassigned,[English],US,1,which country am i at now
...,...,...,...,...,...,...,...,...,...,...,...,...
32855,thread_00024053,24053,2025-05-09 04:23:15.354,2,tie,d14c311dea070f5f0836f7e89983cd4ec64b4d40212ce4...,Analysis,Recommendation,[English],US,1,"Which AI ""Deep Research"" is considered the bes..."
32856,thread_00024053,24053,2025-05-09 04:23:15.354,2,tie,d14c311dea070f5f0836f7e89983cd4ec64b4d40212ce4...,Analysis,Recommendation,[English],US,2,"Doesn't Grok have a ""Deep Research""? How does ..."
32857,thread_00024054,24054,2025-05-09 04:27:18.536,1,model_a,d14c311dea070f5f0836f7e89983cd4ec64b4d40212ce4...,Info Synthesis,Unassigned,[English],US,1,"Which AI ""Deep Research"" is considered the bes..."
32859,thread_00024056,24056,2025-05-09 04:33:32.045,1,model_b,f0746004368a1301223163c1157cd6edc9115a9c6b254d...,Analysis,Factual Lookup,[English],US,1,Give me an analysis based on the Q12025 result...


In [19]:
trump_questions_df = questions_merged_df[questions_merged_df.user_query.str.lower().str.contains('trump')]